In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt

#load data
import data_import as data
from sklearn.model_selection import train_test_split
#import seaborn as sns

#training
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, LeakyReLU, ReLU )


#optimizer
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

C:\Users\woojung\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\woojung\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\woojung\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\woojung\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarnin

In [3]:
print("TensorFlow version: ",tf.__version__)

TensorFlow version:  1.14.0


## 데이터 불러오기
--- 

In [4]:
train = pd.read_csv(os.getcwd()+'/data/train.csv')
test  = pd.read_csv(os.getcwd()+'/data/test.csv')
submission = pd.read_csv(os.getcwd()+'/data/submission.csv')

In [5]:
train.head()

,id,digit,letter,0,1,2,3,4,5,6,...,774,775,776,777,778,779,780,781,782,783
0,1,5,L,1,1,1,4,3,0,0,...,2,1,0,1,2,4,4,4,3,4
1,2,0,B,0,4,0,0,4,1,1,...,0,3,0,1,4,1,4,2,1,2
2,3,4,L,1,1,2,2,1,1,1,...,3,3,3,0,2,0,3,0,2,2
3,4,9,D,1,2,0,2,0,4,0,...,3,3,2,0,1,4,0,0,1,1
4,5,6,A,3,0,2,4,0,3,0,...,4,4,3,2,1,3,4,3,1,2


In [35]:
x_data = train.iloc[:, 3:]
y_data = train.iloc[:, 1]
x_data = np.array(x_data / 255.)
y_data = np.array(y_data)
print(y_data.shape)
print(type(x_data))
print(y_data.shape)
print(type(y_data))

(2048,)
<class 'numpy.ndarray'>
(2048,)
<class 'numpy.ndarray'>


In [7]:
num_classes = 10
img_rows, img_cols, img_ch = 28, 28, 1
input_shape = (img_rows, img_cols, img_ch)

In [25]:
print(x_data.shape)
x_train, x_val, y_train, y_val = train_test_split(
    x_data, y_data, test_size = 0.1)

x_train = x_train.reshape(x_train.shape[0], *input_shape)
x_val = x_val.reshape(x_val.shape[0], *input_shape)
print(x_train.shape)
print(x_val.shape)

(2048, 784)
(1843, 28, 28, 1)
(205, 28, 28, 1)


In [26]:
datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

## 모델 생성
---  

In [28]:
class LeNet5(Model):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.conv1 = Conv2D(6, kernel_size=(5,5), padding='same', activation='relu')
        self.conv2 = Conv2D(16, kernel_size=(5,5), activation='relu')
        self.max_pool = MaxPooling2D(pool_size=(2,2))
        self.flatten = Flatten()
        self.dense1 = Dense(120, activation='relu')
        self.dense2 = Dense(84, activation='relu')
        self.dense3 = Dense(num_classes, activation='softmax')
    def call(self, x):
        x = self.max_pool(self.conv1(x))
        x = self.max_pool(self.conv2(x))
        x = self.flatten(x)        
        x = self.dense3(self.dense2(self.dense1(x)))
        return x

In [29]:
#adam = Adam(lr=5e-4)
# COMPILE WITH ADAM OPTIMIZER AND CROSS ENTROPY COST
model = LeNet5(num_classes)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [30]:
_ = model.predict(x_val[:10])
model.summary()

Model: "le_net5_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            multiple                  156       
_________________________________________________________________
conv2d_5 (Conv2D)            multiple                  2416      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 multiple                  0         
_________________________________________________________________
flatten_2 (Flatten)          multiple                  0         
_________________________________________________________________
dense_6 (Dense)              multiple                  48120     
_________________________________________________________________
dense_7 (Dense)              multiple                  10164     
_________________________________________________________________
dense_8 (Dense)              multiple                  85

In [31]:
# Set a learning rate annealer
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                                patience=3, 
                                verbose=1, 
                                factor=0.2, 
                                min_lr=1e-6)

In [22]:
# Data Augmentation
datagen = ImageDataGenerator(
            rotation_range=10, 
            width_shift_range=0.1, 
            height_shift_range=0.1, 
            zoom_range=0.1)
datagen.fit(x_train)

In [32]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='logs', histogram_freq=1, write_graph=True),
]

In [24]:
for i in range(30):
    history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=32), epochs=80,
          validation_data=(x_val, y_val), verbose=1, callbacks = callbacks)
    print("{} is done".format(i))

Epoch 1/80
58/58 [==============================] - 1s 23ms/step - loss: 2.2917 - acc: 0.1210 - val_loss: 2.2354 - val_acc: 0.1610
Epoch 2/80
58/58 [==============================] - 1s 13ms/step - loss: 2.1789 - acc: 0.1975 - val_loss: 2.0199 - val_acc: 0.2488
Epoch 3/80
58/58 [==============================] - 1s 14ms/step - loss: 2.0665 - acc: 0.2447 - val_loss: 1.9101 - val_acc: 0.2829
Epoch 4/80
58/58 [==============================] - 1s 15ms/step - loss: 1.9510 - acc: 0.2827 - val_loss: 1.7470 - val_acc: 0.3805
Epoch 5/80
58/58 [==============================] - 1s 15ms/step - loss: 1.8620 - acc: 0.3397 - val_loss: 1.5452 - val_acc: 0.4878
Epoch 6/80
58/58 [==============================] - 1s 15ms/step - loss: 1.7595 - acc: 0.3787 - val_loss: 1.4346 - val_acc: 0.4829
Epoch 7/80
58/58 [==============================] - 1s 17ms/step - loss: 1.6721 - acc: 0.4118 - val_loss: 1.3645 - val_acc: 0.5073
Epoch 8/80
58/58 [==============================] - 1s 15ms/step - loss: 1.6026 - a

Exception ignored in: <function TF_Output.<lambda> at 0x000002707EC27C18>
Traceback (most recent call last):
  File "C:\Users\woojung\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 1417, in <lambda>
    __del__ = lambda self: None
KeyboardInterrupt


58/58 [==============================] - 1s 17ms/step - loss: 0.7748 - acc: 0.7303 - val_loss: 1.0362 - val_acc: 0.6780
Epoch 2/80
58/58 [==============================] - 1s 16ms/step - loss: 0.7717 - acc: 0.7379 - val_loss: 0.9450 - val_acc: 0.6878
Epoch 3/80
58/58 [==============================] - 1s 17ms/step - loss: 0.7413 - acc: 0.7254 - val_loss: 0.8877 - val_acc: 0.7171
Epoch 4/80
58/58 [==============================] - 1s 17ms/step - loss: 0.7097 - acc: 0.7509 - val_loss: 0.9909 - val_acc: 0.7122
Epoch 5/80
58/58 [==============================] - 1s 16ms/step - loss: 0.7646 - acc: 0.7450 - val_loss: 0.8993 - val_acc: 0.7122
Epoch 6/80
58/58 [==============================] - 1s 17ms/step - loss: 0.7203 - acc: 0.7564 - val_loss: 0.9897 - val_acc: 0.7024
6 is done
Epoch 1/80
58/58 [==============================] - 1s 15ms/step - loss: 0.7269 - acc: 0.7330 - val_loss: 0.8817 - val_acc: 0.7415
Epoch 2/80
58/58 [==============================] - 1s 18ms/step - loss: 0.6925 - ac

KeyboardInterrupt: 

In [40]:
## Parameter 및 모델 구조 저장
##model.save_weights(f'params.LeNet')
    
#model_json = model.to_json()
#with open(f"Keras_LeNet.json", "w") as json_file : 
#    json_file.write(model_json)

In [66]:
# 예측 진행
X_test = (test[[str(i) for i in range(784)]] / 255.).values.reshape(-1, 28, 28, 1)
#results = model.predict(X_test)
submission['digit']=np.argmax(model.predict(X_test),axis =1)

In [68]:
#submission.digit = results
submission.to_csv('predict_LeNet_v3.csv', index=False)

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# 이건 kfold 코든데 한번 해바바!!
# 내가 뭘잘못했을까ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ

epochs = 50
'''
# Train-Test를 9:1로 분리
X_train2, X_val2, y_train2, y_val2 = train_test_split(
    x_train, y_train, test_size = 0.2)
'''

history = []

#교차검증 수행
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, validation in k_fold.split(x_train, y_train):
    #fit 메서드로 트레이닝 시작
    hist = model.fit(
        x_train[train], y_train[train],
        epochs=epochs,
        validation_data=(x_train[validation], y_train[validation]),  
        verbose=1
    )
    
    history.append(hist)

Train on 1474 samples, validate on 164 samples
Epoch 1/50
1474/1474 [==============================] - 1s 428us/sample - loss: 0.4699 - acc: 0.8426 - val_loss: 0.5104 - val_acc: 0.8354
Epoch 2/50
1474/1474 [==============================] - 1s 430us/sample - loss: 0.4475 - acc: 0.8521 - val_loss: 0.5765 - val_acc: 0.7927
Epoch 3/50
1474/1474 [==============================] - 1s 453us/sample - loss: 0.4709 - acc: 0.8358 - val_loss: 0.6145 - val_acc: 0.7683
Epoch 4/50
1474/1474 [==============================] - 1s 436us/sample - loss: 0.3544 - acc: 0.8948 - val_loss: 0.6090 - val_acc: 0.8049
Epoch 5/50
1474/1474 [==============================] - 1s 516us/sample - loss: 0.3566 - acc: 0.8948 - val_loss: 0.6244 - val_acc: 0.7805
Epoch 6/50
1474/1474 [==============================] - 1s 454us/sample - loss: 0.2311 - acc: 0.9437 - val_loss: 0.6091 - val_acc: 0.7744
Epoch 7/50
1474/1474 [==============================] - 1s 442us/sample - loss: 0.1861 - acc: 0.9552 - val_loss: 0.6558 - val

1474/1474 [==============================] - 1s 437us/sample - loss: 9.4712e-04 - acc: 1.0000 - val_loss: 0.0678 - val_acc: 0.9756
Epoch 18/50
1474/1474 [==============================] - 1s 462us/sample - loss: 9.1377e-04 - acc: 1.0000 - val_loss: 0.0664 - val_acc: 0.9817
Epoch 19/50
1474/1474 [==============================] - 1s 466us/sample - loss: 8.1859e-04 - acc: 1.0000 - val_loss: 0.0666 - val_acc: 0.9817
Epoch 20/50
1474/1474 [==============================] - 1s 438us/sample - loss: 7.5268e-04 - acc: 1.0000 - val_loss: 0.0652 - val_acc: 0.9817
Epoch 21/50
1474/1474 [==============================] - 1s 461us/sample - loss: 7.0696e-04 - acc: 1.0000 - val_loss: 0.0651 - val_acc: 0.9817
Epoch 22/50
1474/1474 [==============================] - 1s 459us/sample - loss: 6.8181e-04 - acc: 1.0000 - val_loss: 0.0640 - val_acc: 0.9817
Epoch 23/50
1474/1474 [==============================] - 1s 474us/sample - loss: 6.3302e-04 - acc: 1.0000 - val_loss: 0.0638 - val_acc: 0.9817
Epoch 24/50

1474/1474 [==============================] - 1s 451us/sample - loss: 2.2026e-04 - acc: 1.0000 - val_loss: 0.0230 - val_acc: 0.9939
Epoch 32/50
1474/1474 [==============================] - 1s 453us/sample - loss: 2.1833e-04 - acc: 1.0000 - val_loss: 0.0230 - val_acc: 0.9939
Epoch 33/50
1474/1474 [==============================] - 1s 462us/sample - loss: 2.0354e-04 - acc: 1.0000 - val_loss: 0.0228 - val_acc: 0.9939
Epoch 34/50
1474/1474 [==============================] - 1s 462us/sample - loss: 1.9178e-04 - acc: 1.0000 - val_loss: 0.0226 - val_acc: 0.9939
Epoch 35/50
1474/1474 [==============================] - 1s 507us/sample - loss: 1.8286e-04 - acc: 1.0000 - val_loss: 0.0225 - val_acc: 0.9939
Epoch 36/50
1474/1474 [==============================] - 1s 475us/sample - loss: 1.7482e-04 - acc: 1.0000 - val_loss: 0.0225 - val_acc: 0.9939
Epoch 37/50
1474/1474 [==============================] - 1s 483us/sample - loss: 1.6702e-04 - acc: 1.0000 - val_loss: 0.0223 - val_acc: 0.9939
Epoch 38/50

1474/1474 [==============================] - 1s 462us/sample - loss: 7.5051e-05 - acc: 1.0000 - val_loss: 0.0069 - val_acc: 1.0000
Epoch 46/50
1474/1474 [==============================] - 1s 475us/sample - loss: 7.2296e-05 - acc: 1.0000 - val_loss: 0.0070 - val_acc: 1.0000
Epoch 47/50
1474/1474 [==============================] - 1s 453us/sample - loss: 6.9748e-05 - acc: 1.0000 - val_loss: 0.0071 - val_acc: 1.0000
Epoch 48/50
1474/1474 [==============================] - 1s 463us/sample - loss: 6.7327e-05 - acc: 1.0000 - val_loss: 0.0071 - val_acc: 1.0000
Epoch 49/50
1474/1474 [==============================] - 1s 448us/sample - loss: 6.5066e-05 - acc: 1.0000 - val_loss: 0.0071 - val_acc: 1.0000
Epoch 50/50
1474/1474 [==============================] - 1s 478us/sample - loss: 6.2733e-05 - acc: 1.0000 - val_loss: 0.0071 - val_acc: 1.0000
Train on 1474 samples, validate on 164 samples
Epoch 1/50
1474/1474 [==============================] - 1s 464us/sample - loss: 0.0633 - acc: 0.9851 - val_

Epoch 8/50
1475/1475 [==============================] - 1s 477us/sample - loss: 4.1080e-05 - acc: 1.0000 - val_loss: 3.0427e-05 - val_acc: 1.0000
Epoch 9/50
1475/1475 [==============================] - 1s 470us/sample - loss: 3.8921e-05 - acc: 1.0000 - val_loss: 2.9730e-05 - val_acc: 1.0000
Epoch 10/50
1475/1475 [==============================] - 1s 485us/sample - loss: 3.7028e-05 - acc: 1.0000 - val_loss: 2.9205e-05 - val_acc: 1.0000
Epoch 11/50
1475/1475 [==============================] - 1s 485us/sample - loss: 3.5314e-05 - acc: 1.0000 - val_loss: 2.8537e-05 - val_acc: 1.0000
Epoch 12/50
1475/1475 [==============================] - 1s 464us/sample - loss: 3.3715e-05 - acc: 1.0000 - val_loss: 2.8024e-05 - val_acc: 1.0000
Epoch 13/50
1475/1475 [==============================] - 1s 464us/sample - loss: 3.2259e-05 - acc: 1.0000 - val_loss: 2.7386e-05 - val_acc: 1.0000
Epoch 14/50
1475/1475 [==============================] - 1s 478us/sample - loss: 3.0920e-05 - acc: 1.0000 - val_loss: 2.

In [21]:
score = model.evaluate(x_val, y_val, batch_size=32)

6/6 [==============================] - 0s 6ms/step - loss: 0.0043 - accuracy: 1.0000


In [60]:
score

[4.436415672302246, 0.997560977935791]